## **Project One: TripAdvisor Recommendation Challenge - Beating BM25**
_**Authors:** Alberto MARTINELLI, Alessia SARRITZU_

The goal of the project is to develop a recommendation system that relies solely on user reviews to suggest similar places based on given queries. The system will propose the most relevant location based on the text of the reviews.

1. **Data pre-processing**:
  - Utilise only the reviews where ratings are composed strictly with this aspects on reviews with ratings for the following aspects: **service**, **cleanliness**, **overall**, **value**, **location**, **sleep quality**, and **rooms**.
  - Concatenate reviews by `offering_id` to compute average ratings for evaluation.

2. **BM25 Implementation**:
    - Implement a BM25 baseline using the **Rank-BM25** library.
    - Measure the performance of BM25 through Mean Square Error (MSE) between each ratings of the query place and the recommended place.

3. **Enhanced Unsupervised Model**:
    - Create a new unsupervised model to outperform BM25, potentially integrating it with other methods, while ensuring the model does not directly utilize ratings in its learning process.
    - Measure performance of the **Enhanced Model** through Mean Square Error (MSE) between the ratings of the query place and the recommended place, with the aim of achieving a lower MSE than the BM25 baseline.

### 1)

In [19]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
df = pd.read_csv('reviews.csv')
df.sample(1)

,ratings,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile
784146,{'overall': 4.0},“Bring earplugs and you'll be just fine”,"I thought this hotel was pretty decent for the price. Stay here if you're looking for a cheap but reasonable stay. The main problem is the noise. Fedex is based in Memphis, and I think the road the hotel is on carries a lot of their freight traffic. And the nearby airport doesn't help either. Request a room in the back; my friends stayed on the other side and they said it was quieter. \nBesides the noise, though, we had a fine stay. The room seemed very clean and came with a microwave and large fridge. Also a couch and table. The water pressure in the shower was very nice. One of my traveling companions said it was the best shower he'd had in years. Breakfast is above average -- hardboiled eggs (unpeeled!), bread, bagels, cream cheese, apples, bananas, sausage biscuits, hashbrowns (tasted a little freezer-y), pastries. I didn't use the fitness center but it looked very nice -- bike, treadmill, elliptical, and a few free weights. Staff was very friendly, particularly the guy setting out the breakfast. He also seemed very hygiene conscious which I really appreciated.\nMy only real complaint was there were very few vending options on site. I wanted a bottle of water, but the lone drink machine was empty. There was one rather lame snack machine, too. However, there is a huge Exxon gas station/market about a block away that's open 24 hours a day during the week (closes at 10 Fri/Sat), so I should have just gone down there. Other than the palatial Exxon and the endless parade of trucks passing by, the neighborhood seemed pretty desolate. Not unsafe, just dull.","{'username': 'tokoloshgrrl', 'num_cities': 18, 'num_helpful_votes': 19, 'num_reviews': 20, 'num_type_reviews': 19, 'id': '6015E940245217F8090BA86A7C26F665', 'location': ''}",March 2008,239950,0,2008-03-31,14686590,False


On the table Reviews.csv, keep only the reviews where ratings are composed strictly with this 
aspects:  
“service”, “cleanliness”, “overall”, “value”, “location”, “sleep quality”, “rooms” 
(not more and not less in order to compare places accurately)

In [20]:
required_aspects = {"service", "cleanliness", "overall", "value", "location", "sleep_quality", "rooms"}

filtered_df = df[df['ratings'].apply(lambda x: set(eval(x).keys()) == required_aspects)]

df = filtered_df

df.sample(1)

,ratings,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile
582102,"{'service': 1.0, 'cleanliness': 4.0, 'overall': 2.0, 'value': 3.0, 'location': 5.0, 'sleep_quality': 3.0, 'rooms': 4.0}","“Be Careful what you wish for, if it's service.....”","Started off pretty good, until I had a small problem. The television failed to work, so I called down to the office who said the maintenance would be back the next morning (Tuesday). As of Friday nothing had been fixed so I gave up on it. This led me to the other things in the room that might be a little bit of a put off. The curtains do not even cover half of the windows leaving the blinds to do all the work in the privacy department which were broken on one side.\nPrivacy side apart there were other minor items that were a bit bothersome. Only one of the lamps worked, no remote to try the tv again, but the room next door had one...? The sound barrier between the rooms was atrociously terrible. I was woken up 4 or 5 times on consecutive nights by a woman who coughed as loud as possible. One morning the Fire alarm burst out of nowhere , proceeding to the lobby the lady at the desk (hardly speaks English) tried to inform me that they couldn't locate the area it was in or for what matter the alarms went off, causing concern of safety. I guess what I am trying to say is that the service department really is lacking and that it seems as if the cultural barrier really influences how service is provided.\nThe city sounds don't bother me at all, I actually appreciate them versus the country setting I am used to. All together the price and the place really make most of the difference here. It is just about 2 blocks from the Westfield shopping center which boasts 200 shops. It's in a clean part of the city near any sort of food or activity you could want. If you prefer quiet nights this might not be the best place for you because it is located right next to downtown Market. \nAs far as the checklist on the top of the page, I would say that the cleanliness is very good. Every morning I left I came back and it smelled fresh and the bed was made etc. The rooms however were set up as if it were a dorm room type with bright paintings and neon yellow and pink comforters as well as a flower shaped mirror. I am not 100% sure on the value as I am splitting with 4 people and do not know the final price for the week that I am staying here but for the location and what you get I rated it a 3/5.\nAll in all, be careful what you wish for, especially if it is service because it might be hit or miss with this place. I am a stickler for service because in my mind it can really make or break a place. If I am paying you to set me up in a room I expect a certain amount of service for what I am paying. (A working TV would help, maybe Breakfast, and definitely curtains and working blinds.) \nThanks for reading and enjoy your future trips, I hope this helped give you an idea what this place was like, at least from my point of view.","{'username': 'LaFaverL', 'num_reviews': 1, 'id': '135FC8FD519810EB26405339535D71D1', 'location': 'Blacksburg, Virginia'}",February 2011,80771,0,2011-02-24,97862946,False


You must concatenate reviews from the same place based on attribute “offering_id”. The rating of a 
place is just the average of all the reviews ratings on each aspect.

In [21]:
filtered_df['ratings'] = filtered_df['ratings'].apply(eval)
expanded_ratings_df = pd.json_normalize(filtered_df['ratings']).join(filtered_df[['offering_id', 'title', 'text']])

# Calculate the mean of each rating aspect and concatenate reviews
final_df = expanded_ratings_df.groupby('offering_id').agg(
    rating_mean=('service', 'mean'),  # Average rating for 'service' aspect (you can add others as needed)
    cleanliness_mean=('cleanliness', 'mean'),
    overall_mean=('overall', 'mean'),
    value_mean=('value', 'mean'),
    location_mean=('location', 'mean'),
    sleep_quality_mean=('sleep_quality', 'mean'),
    rooms_mean=('rooms', 'mean'),
    reviews=('text', lambda x: ' '.join(x)), # Concatenate all text entries
    titles=('title', lambda x: ' '.join(x))  # Concatenate all title entries
).reset_index()


final_df['text'] = final_df['titles'] + " " + final_df['reviews']
final_df = final_df.drop(columns=['titles', 'reviews'])
final_df.sample(1)


offering_id  rating_mean  cleanliness_mean  overall_mean  value_mean  \
273      93464.0     4.049674          4.178339      3.983713     3.87785   

     location_mean  sleep_quality_mean  rooms_mean  \
273       4.605049            4.093648    3.871336   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

### 2)

In [22]:
# Select a random query place from the dataset
query_place = df.sample(1).iloc[0]
query_id = query_place['offering_id']
query_ratings = query_place[['service', 'cleanliness', 'overall', 'value', 'location', 'sleep_quality', 'rooms']]
query_text = query_place['text']

print("Query Place Details:")
print(f"Offering ID: {query_id}")
print("Ratings:")
for aspect, rating in query_ratings.items():
    print(f"  {aspect}: {rating}")
print("\nReview Text:")
print(query_text)

# Exclude the query place from the documents to avoid recommending it
documents_df = df[df['offering_id'] != query_id].reset_index(drop=True)


KeyError: "None of [Index(['service', 'cleanliness', 'overall', 'value', 'location',\n       'sleep_quality', 'rooms'],\n      dtype='object')] are in the [index]"

In [ ]:
from rank_bm25 import BM25Okapi
from sklearn.metrics import mean_squared_error

# Load and preprocess the reviews (assuming df contains concatenated reviews by 'offering_id')
documents = documents_df['text'].apply(lambda x: x.split())  # Tokenize each document by splitting words
bm25 = BM25Okapi(documents)

# Define a query (a review or set of reviews from a specific place)
query = "The room was clean and the location was great".split()  # Tokenized query

# Get BM25 scores for the query across all documents
scores = bm25.get_scores(query)

# Step 5: Retrieve the top matching place based on BM25 score
top_n = 1  # Number of top matches to retrieve
top_match_index = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_n][0]
top_match = documents_df.iloc[top_match_index]

# Step 6: Calculate the MSE between the ratings of the query place and the recommended place
recommended_ratings = top_match[['service', 'cleanliness', 'overall', 'value', 'location', 'sleep_quality', 'rooms']]
mse = mean_squared_error(query_ratings, recommended_ratings)

# Print the results
print(f"Query Offering ID: {query_id}")
print(f"Recommended Offering ID: {top_match['offering_id']}")
print(f"BM25 Score: {scores[top_match_index]}")
print(f"MSE: {mse}")